In [4]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
#import urdf2casadi.urdf2casadi.urdfparser
#import urdf2casadi.urdf2casadi.geometry.plucker as plucker
import numpy as np
import PyKDL as kdl
#import kdl_parser.kdl_parser_py.kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl

## Load models:

In [2]:
#ok, ur_tree = kdlurdf.treeFromFile('./urdf2casadi/examples/urdf/test.urdf')
ok, ur_tree = kdlurdf.treeFromFile('ur5_rbdl.urdf')
asd = urdf2casadi.urdf2casadi.urdfparser.URDFparser()
#robot_desc = asd.from_file("./urdf2casadi/examples/urdf/test.urdf")
robot_desc = asd.from_file("ur5_rbdl.urdf")
root = 'base_link'
tip = 'wrist_3_link'
ur_chain = ur_tree.getChain(root,tip)
#rbdlmodel = rbdl.loadModel("ur5_rbdl.urdf")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
ur5_rbdl.urdf


## Declare variables to use in timing tests:

In [3]:
jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = len(jlist)
print njoints
print len(qmax)

H_ur = kdl.JntSpaceInertiaMatrix(6)

grav = kdl.Vector()
grav[2] = -9.81

q = kdl.JntArray(njoints)
qdot = kdl.JntArray(njoints)

q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)

q_sx = cs.SX.zeros(njoints)
print q_sx
qdot_sx = cs.SX.zeros(njoints)

q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints

tau_kdl = kdl.JntArray(njoints)
grav_kdl = kdl.JntArray(njoints)

nitr = 50000
nrepeat = 10



6
6
@1=0, [@1, @1, @1, @1, @1, @1]


# Timing Coriolis
## Coriolis KDL timing:

In [4]:
def C_kdl():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(ur_chain, grav).JntToCoriolis(q, qdot, tau_kdl)
#kdl_time = [None]*30

#timeit_C_kdl = timeit("C_kdl()", setup = "from __main__ import C_kdl", number = nitr)
#print timeit_C_kdl
#print timeit_C_kdl/nitr
timeit_C_kdl_repeat = repeat("C_kdl()", setup = "from __main__ import C_kdl", repeat = nrepeat, number = nitr)
print timeit_C_kdl_repeat

[1.2641470432281494, 1.227410078048706, 1.1931788921356201, 1.1927030086517334, 1.1964430809020996, 1.19508695602417, 1.1952321529388428, 1.2077960968017578, 1.1975998878479004, 1.194835901260376]


## Coriolis u2c timing:

With timeit magic:

In [5]:
C_u2c = asd.get_jointspace_bias_matrix(root, tip)

base_link
revolute
shoulder_link
revolute
upper_arm_link
revolute
forearm_link
revolute
wrist_1_link
revolute
wrist_2_link
revolute
wrist_3_link


With timeit module:

In [6]:
def C_u2c_func():    
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    C_u2c(q_none, qdot_none)

timeit_C_u2c = timeit("C_u2c_func()", setup = "from __main__ import C_u2c_func", number = nitr)
print timeit_C_u2c
print timeit_C_u2c/nitr
timeit_C_u2c_repeat = repeat("C_u2c_func()", setup = "from __main__ import C_u2c_func", repeat = nrepeat, number = nitr)
print timeit_C_u2c_repeat

#1.68 as best 

#%store timeit_C_u2c_repeat

2.41834998131
4.83669996262e-05
[2.357588052749634, 2.341248035430908, 2.3379530906677246, 2.3332231044769287, 2.422407865524292, 2.4123408794403076, 2.333911895751953, 2.33134388923645, 2.331494092941284, 2.416335105895996]


In [7]:
tau_rbdl = np.zeros(njoints)

In [11]:
def C_rbdl():   
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, tau_rbdl)

#timeit_C_rbdl = timeit("C_rbdl()", setup = "from __main__ import C_rbdl", number = 50000)
#print timeit_C_rbdl
#print timeit_C_rbdl/50000
timeit_C_rbdl_repeat = repeat("C_rbdl()", setup = "from __main__ import C_rbdl", repeat = 3, number = 50000)
print timeit_C_rbdl_repeat
#1.91 as best 

[2.0779550075531006, 1.9433138370513916, 2.047614812850952]


In [9]:
print "minimum u2c coriolis:", min(timeit_C_u2c_repeat)
print "minimum kdl coriolis:", min(timeit_C_kdl_repeat)
    

minimum u2c coriolis: 1.68841195107
minimum kdl coriolis: 1.01719594002
